## Live Update In-Progress

In [ ]:
import numpy as np
np.argsort()

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'NONE'
sample_size = 5

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
%%bash -s "$PLATFORM"
cd ../../hardware/victims/firmware/simpleserial-glitch
make PLATFORM=$1 CRYPTO_TARGET=NONE

In [ ]:
cw.program_target(scope, prog, "../../hardware/victims/firmware/simpleserial-glitch/simpleserial-glitch-{}.hex".format(PLATFORM))

In [ ]:
import logging

#debug - delete this 💩

target.flush()
target.write("g\n")
r = target.simpleserial_read_witherrors("r", 4)
print(r)


scope.io.nrst = False
scope.io.nrst = True

r = target.simpleserial_read_witherrors("r", 1)
print(r)

In [ ]:
#debug - delete this 💩

from ipywidgets import interact, widgets
from IPython.display import display
import time

text = widgets.Text(
    value='cA3',
    placeholder='cA3',
    description='Serial TX',
    disabled=False
)
display(text)

def callback(wdgt):
    sent = wdgt.value + "\n"
    target.write(sent)
    print(sent)

text.on_submit(callback)

In [ ]:
import time

#debug - delete this 💩

target.read(100)

scope.io.nrst = False
time.sleep(0.1)

scope.io.target_pwr = False
time.sleep(0.5)
scope.io.target_pwr = True
time.sleep(1)


scope.io.nrst = None
time.sleep(1.5)

target.read(100)

In [ ]:
#Basic setup
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"

scope.io.hs2 = "glitch"  
print(scope.glitch)

In [ ]:
def reboot_flush():            
    scope.io.nrst = False
    time.sleep(0.1)
    scope.io.nrst = True
    time.sleep(0.1)
    #Flush garbage too
    target.read(100)

In [ ]:
# GlitchController will be part of ChipWhisperer core - just run this block
# for now.

%matplotlib notebook
import matplotlib.pylab as plt

import ipywidgets as widgets          
import chipwhisperer.common.results.glitch as glitch

class GlitchController(object):
    
    def __init__(self, groups, parameters):
        self.groups = groups
        self.parameters = parameters
        
        self.results = glitch.GlitchResults(groups=groups, parameters=parameters)
        
        self.parameter_min = [0.0] * len(parameters)
        self.parameter_max = [10.0] * len(parameters)
        self.steps = [1]
        
        self.widget_list_parameter = None
        self.widget_list_groups = None
        
        self.clear()
        
    def clear(self):
        self.results.clear()        
        self.group_counts = [0] * len(self.groups)
        
        if self.widget_list_groups:
            for w in self.widget_list_groups:
                w.value = 0
        
    def set_range(self, parameter, low, high):
        
        if high < low:
            t = low
            low = high
            high = t
        
        i = self.parameters.index(parameter)
        self.parameter_min[i] = low
        self.parameter_max[i] = high
        
        if self.widget_list_parameter:
            # When changing them, need to ensure we don't have min > max ever or will throw
            # an error, so we set max to super-high first.
            self.widget_list_parameter[i].max = 1E9
            self.widget_list_parameter[i].min = low
            self.widget_list_parameter[i].max = high
    
    def set_step(self, parameter, step):
        '''Set size, can be either a single value or a list of step-sizes to iterate through'''
        raise NotImplementedError("TODO")
        
    def set_global_step(self, steps):
        '''Set step for all parameters (Eventually support per-parameter)'''
        if hasattr(steps, "__iter__"):
            self.steps = steps
        else:
            self.steps = [steps]
    
    def add(self, group, parameters, strdesc=None, metadata=None):
        self.results.add(group, parameters, strdesc, metadata)    
        
        i = self.groups.index(group)        
        #Basic count
        self.group_counts[i] += 1
        self.widget_list_groups[i].value =  self.group_counts[i]
    
    def display_stats(self):
        
        self.widget_list_groups = [widgets.IntText(value=0, description=group + " count:", disabled=True)
                                   for group in self.groups]
        
        self.widget_list_parameter = [widgets.FloatSlider(
                                            value=self.parameter_min[i],
                                            min=self.parameter_min[i],
                                            max=self.parameter_max[i],
                                            step=0.01,
                                            description=p + " setting:",
                                            disabled=True,
                                            continuous_update=False,
                                            orientation='horizontal',
                                            readout=True,
                                            readout_format='.01f')
                                          for i,p in enumerate(self.parameters)]
            
        display(*(self.widget_list_groups + self.widget_list_parameter))
       
        
    def glitch_values(self, clear=True):
        """Generator returning the given parameter values in order, using the step size (or step list)"""
        
        self.parameter_values = self.parameter_min[:]
        
        if clear:
            self.clear()
        
        for stepsize in self.steps:
            for val in self._loop_rec(0, len(self.parameter_values)-1, stepsize):
                if self.widget_list_parameter:
                    for i,v in enumerate(val):
                        self.widget_list_parameter[i].value = v
                yield val
        
    def _loop_rec(self, parameter_index, final_index, step):
        self.parameter_values[parameter_index] = self.parameter_min[parameter_index]
        if parameter_index == final_index:            
            while self.parameter_values[parameter_index] <= self.parameter_max[parameter_index]:                                
                yield self.parameter_values
                self.parameter_values[parameter_index] += step
        else:
            while self.parameter_values[parameter_index] <= self.parameter_max[parameter_index]: 
                yield from self._loop_rec(parameter_index+1, final_index, step)
                self.parameter_values[parameter_index] += step

        
gc = GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset"])
gc.display_stats()
#gc.display_controls()


fig = plt.figure()
plt.plot(-48, 48, ' ')
plt.plot(48, -48, ' ')
plt.plot(-48, -48, ' ')
plt.plot(48, 48, ' ')
#plt.ion()


Ready to rock - run the following block which should update the stats above. Note you need to run stop on the block to stop anything... sometimes need to hit it a bunch of times. TOOD is adding a button to abort.

In [ ]:


from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm import tnrange
import struct

scope.glitch.ext_offset = 0

#Uncomment for the full sweep
#gc.set_range("width", -48, 48)
#gc.set_range("offset", -48, 48)
#gc.set_global_step([4.0, 0.8])
#scope.glitch.repeat = 50

#Uncomment for upper left region
gc.set_range("width", -48, 48)
gc.set_range("offset", -48, 48)
gc.set_global_step([8.0, 4.0, 2.0, 1.0])
scope.glitch.repeat = 8 #4

#Uncomment for more focused region
#gc.set_range("width", 7, 8)
#gc.set_range("offset", -24, -22)
#gc.set_global_step([4.0, 0.8, 0.25, 0.1])
#scope.glitch.repeat = 5

sample_size = 1

for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    for i in tnrange(sample_size, leave=False):
        scope.arm()
        #reset_target(scope)
        #target.write("cA2\n") #send A2 is CORRECT
        #target.write("cA3\n") #send A3 is INCORRECT
        
        #Do glitch loop
        target.write("g\n")
        
        ret = scope.capture()
        
        
        #val = target.simpleserial_read_witherrors('r', 1)#For code-flow check
        
        
        val = target.simpleserial_read_witherrors('r', 4)#For loop check
        
        if ret:
            print('Timeout - no trigger')
            gc.add("reset", (scope.glitch.width, scope.glitch.offset))
            plt.plot(scope.glitch.width, scope.glitch.offset, 'xr', alpha=1)
            fig.canvas.draw()

            #Device is slow to boot?
            reboot_flush()

        else:
            if val['valid'] is False:
                gc.add("reset", (scope.glitch.width, scope.glitch.offset))
                print("x")
            else:
                
                #print(val['payload'])
                
                #gcnt = struct.unpack("<b", val['payload'])[0] #for code-flow check
                gcnt = struct.unpack("<I", val['payload'])[0]

                #print(gcnt)                
                # for table display purposes
                #if gnt != 0: #for code-flow check
                if gcnt != 2500: #for loop check
                    gc.add("success", (scope.glitch.width, scope.glitch.offset))
                    plt.plot(scope.glitch.width, scope.glitch.offset, '+g')
                    print(val['payload'])
                    print("🐙", end="")
                else:
                    gc.add("normal", (scope.glitch.width, scope.glitch.offset))
                    plt.plot(scope.glitch.width, scope.glitch.offset, '.k', alpha=0.1)
                    fig.canvas.draw()
                    

print("Done glitching")

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

# plot_2d is part of CW core, but has some bugs with huge data (like generated here) where you don't
# want to plot everything. For now just use the following which replaces the plot_2d function with some bug-fixes

def plot_2d(self, plotdots, x_index=0, y_index=1, x_units=None, y_units=None, mask=True):
    '''
    Generate a 2D plot of glitch success rate using matplotlib.

    Plotting is done in the default figure - you may need to call plt.figure() before and
    plt.show() after calling this function if you want more control (or the figure does
    not show by default).
    '''

    data = self.calc()

    #We only want legend to show for first element... bit of a hack here
    legs = self.groups[:]

    #Generate success rates
    for p in data:
        #Plot based on non-zero priority if possible
        for g in self.groups:
            if plotdots[g]:
                if p[g] > 0:
                    if g in legs:
                        leg = g.title()
                        #No need to show this one anymore
                        legs.remove(g)
                    else:
                        leg = None

                    sr = float(p[g]) / float(p['_total']) + 0.5
                    
                    if sr > 1.0:
                        sr = 1.0
                    
                    plt.plot(p['_parameter'][x_index], p['_parameter'][y_index], plotdots[g], alpha=sr, label=leg)

                    if mask:
                        break

    xlabel = self.parameters[x_index].title()
    if x_units:
        xlabel += " (" + x_units + ")"
    plt.xlabel(xlabel)

    ylabel = self.parameters[y_index].title()
    if y_units:
        ylabel += " (" + y_units + ")"
    plt.ylabel(ylabel)

    plt.legend()

# Plot the results. If you have no glitches or resets this will generat an error -
# this is expected behaviour as we also tell it not to plot the "normal" responses.
plot_2d(gc.results, plotdots={"success":"+g", "reset":"xr", "normal":None})

In [ ]:
scope.io.target_pwr = True

In [ ]:
#width -48 to -30
#offset 40 to 48

In [ ]:
%%bash
python -m jupyter nbextension enable --py widgetsnbextension

In [ ]:
scope.adc.trig_count